In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Football/"

In [ ]:
# Wczytanie danych z pliku CSV
data = pd.read_csv(path +'fianl_data_for_neural_network_train_3.csv', sep=',')

In [ ]:
data

In [ ]:
data = data.drop(columns=['game_date', 'home_team', 'away_team'])

In [ ]:
# data = data[[
# 'home_team_last_5_search_team_summary_xg',
# 'home_team_last_5_search_team_summary_gca',
# 'home_team_last_5_search_team_passes_passes_into_final_third',
# 'home_team_last_5_search_team_defensive_blocked_shots',
# 'home_team_last_5_opponent_team_summary_goals',
# 'home_team_last_5_opponent_team_summary_xg',
# 'home_team_last_5_opponent_team_defensive_blocked_shots',
# 'home_team_last_5_result',
# 'away_team_last_5_search_team_summary_xg',
# 'away_team_last_5_search_team_summary_gca',
# 'away_team_last_5_search_team_passes_passes_into_final_third',
# 'away_team_last_5_search_team_defensive_blocked_shots',
# 'away_team_last_5_opponent_team_summary_goals',
# 'away_team_last_5_opponent_team_summary_xg',
# 'away_team_last_5_opponent_team_defensive_blocked_shots',
# 'away_team_last_5_result',
# 'home_team_elo',
# 'away_team_elo',
# 'result'
# ]]

In [ ]:
# zmiana w result z 3 na 2

data['result'] = data['result'].replace(3.0, 2.0)

In [ ]:
# Podział danych na funkcje (X) i etykiety (y)
X = data.drop('result', axis=1)
y = data['result']

# Podział danych na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizacja danych za pomocą StandardScaler
scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)


In [ ]:
# One-hot encoding dla etykiet
num_classes = 3
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

model = Sequential()


# Add Dense layers with Batch Normalization and Dropout
model.add(Dense(18, input_shape=(X_train_normalized.shape[1],), activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

# Add EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_normalized, y_train_categorical,
          epochs=150,
          batch_size=128,
          validation_split=0.2,  # Use a validation split
          callbacks=[early_stopping],  # Add early stopping callback
          verbose=1)

# Evaluate the model on the test set
score = model.evaluate(X_test_normalized, y_test_categorical)
print('Test loss:', score[0])
print('Test accuracy:', score[1])